In [1]:
import sys
import os

# Add parent directory to sys.path
sys.path.append(os.path.abspath(".."))


In [2]:
from models.spin_system import SpinSystem

import pandas as pd
import numpy as np
import tensorflow as tf

import plotly.express as px
import matplotlib.pyplot as plt


In [3]:


def generate_coupling_tensor(D, L, J0=10, alpha=1):
    """
    Vectorized coupling tensor for arbitrary D-dimensional lattice with decaying interactions.
    Tensor shape: (L,)*D + (L,)*D
    J[i1,...,iD,j1,...,jD] = J0 * exp(-alpha * Euclidean distance)
    """
    N = L ** D

    # Generate all D-dimensional coordinates (N x D)
    coords = np.array(np.meshgrid(*[np.arange(L)]*D, indexing='ij')).reshape(D, -1).T

    # Compute pairwise differences: (N, N, D)
    diff = coords[:, None, :] - coords[None, :, :]
    # Euclidean distance
    distances = np.linalg.norm(diff, axis=2)

    # Compute decaying interactions
    J_flat = J0 * np.exp(-alpha * distances)
    np.fill_diagonal(J_flat, 0)  # no self-interaction

    # Map flattened indices to tensor shape
    tensor_shape = (L,)*D*2
    J_tensor = np.zeros(tensor_shape)

    # Vectorized assignment: convert flat indices to multi-indices
    multi_idx = [np.unravel_index(np.arange(N), (L,)*D) for _ in range(2)]
    # multi_idx = [[i1_array,...,iD_array], [j1_array,...,jD_array]]? We'll iterate below

    for i in range(N):
        for j in range(N):
            J_tensor[tuple(coords[i]) + tuple(coords[j])] = J_flat[i,j]

    return J_tensor


def generate_nn_coupling_tensor(D, L):
    """
    Vectorized nearest-neighbor coupling tensor for arbitrary D-dimensional lattice.
    Tensor shape: (L,)*D + (L,)*D
    J[i1,...,iD,j1,...,jD] = 1 if Manhattan distance = 1, else 0
    """
    N = L ** D

    # Generate all coordinates
    coords = np.array(np.meshgrid(*[np.arange(L)]*D, indexing='ij')).reshape(D, -1).T

    # Pairwise differences
    diff = coords[:, None, :] - coords[None, :, :]
    manhattan_dist = np.abs(diff).sum(axis=2)

    # Only nearest neighbors
    nn_mask = (manhattan_dist == 1)

    # Map to tensor
    tensor_shape = (L,)*D*2
    J_tensor = np.zeros(tensor_shape, dtype=int)

    for i in range(N):
        for j in range(N):
            if nn_mask[i,j]:
                J_tensor[tuple(coords[i]) + tuple(coords[j])] = 1

    return J_tensor

def generate_periodic_nn_coupling_tensor(D, L):
    """
    Vectorized nearest-neighbor coupling tensor with periodic boundaries.
    J[i1,...,iD,j1,...,jD] = 1 if periodic Manhattan distance = 1, else 0
    """
    # Generate all coordinates: shape (N, D)
    coords = np.array(np.meshgrid(*[np.arange(L)]*D, indexing='ij')).reshape(D, -1).T
    N = coords.shape[0]

    # Compute pairwise differences with broadcasting
    diff = np.abs(coords[:, None, :] - coords[None, :, :])
    
    # Apply periodic boundary
    diff = np.minimum(diff, L - diff)

    # Manhattan distance
    manhattan_dist = diff.sum(axis=2)

    # Nearest neighbors mask
    nn_mask = (manhattan_dist == 1)

    # Create empty tensor and set neighbors
    J_tensor = np.zeros((L,)*D*2, dtype=np.float32)
    
    # Get indices where nn_mask is True
    idx_i, idx_j = np.nonzero(nn_mask)
    
    # Set values in the tensor
    for i, j in zip(idx_i, idx_j):
        J_tensor[tuple(coords[i]) + tuple(coords[j])] = 1.0

    return J_tensor



In [ ]:

def run_temperature_sweep(D, L, betas, J=None, steps=None, dynamic_theta = True, theta_max=0.1):
    results = []
    if steps == None:
        steps = L**D

    for beta in betas:
        
        system = SpinSystem(D, L, J=J, keep_history=True)
        if dynamic_theta == True:
            theta_max = theta_max/beta
        print(f"Sweeping with beta {beta} with max rotation of {theta_max} radian")
        system.metropolis_sweep(beta, steps=steps, theta_max=theta_max)

        df = pd.DataFrame(system.history.get()).T
        df["beta"] = beta
        results.append(df)

    return pd.concat(results, ignore_index=True)


def plot_tensor_lattice(sigma: tf.Tensor, J: tf.Tensor):
    """
    Plot a 2D spin lattice with node values from sigma and
    edge thickness from J (only nearest neighbors for clarity).
    """
    sigma_np = sigma.numpy()
    Lx, Ly = sigma_np.shape

    fig, ax = plt.subplots(figsize=(6,6))

    # Plot nodes
    for i in range(Lx):
        for j in range(Ly):
            ax.scatter(i, j, c=[[sigma_np[i,j]]], cmap='coolwarm', s=500, edgecolors='k')
            ax.text(i, j, f"{sigma_np[i,j]:.2f}", color='black',
                    ha='center', va='center', fontsize=10)

    # Plot edges: only nearest neighbors for clarity
    for i in range(Lx):
        for j in range(Ly):
            for di,dj in [(0,1),(1,0)]:  # right and down neighbors
                ni, nj = i+di, j+dj
                if ni < Lx and nj < Ly:
                    weight = J[i,j,ni,nj].numpy()
                    ax.plot([i, ni], [j, nj],
                            'k-', alpha=0.6, linewidth=0.5 + 3*abs(weight)/np.max(np.abs(J.numpy())))

    ax.set_xticks(range(Lx))
    ax.set_yticks(range(Ly))
    ax.set_xlim(-0.5, Lx-0.5)
    ax.set_ylim(-0.5, Ly-0.5)
    ax.set_aspect('equal')
    ax.set_title("2D Spin Lattice (Tensor Form)")
    plt.show()

# model = SpinSystem(2, 10)
# plot_tensor_lattice(model.sigma, tf.convert_to_tensor(J, dtype=tf.float32))

In [5]:
D = 2
L = 100
N = L ** D


J = generate_periodic_nn_coupling_tensor(D, L)

In [ ]:
betas = [0.1, 1, 10, 100]

df = run_temperature_sweep(D, L, betas, steps=1000, dynamic_theta=False, theta_max=np.pi)

Sweeping with beta:  0.1
Metropolis sweep: 511/1000 accepted (51.10%)
Sweeping with beta:  1
Metropolis sweep: 488/1000 accepted (48.80%)
Sweeping with beta:  10
Metropolis sweep: 451/1000 accepted (45.10%)
Sweeping with beta:  100
Metropolis sweep: 503/1000 accepted (50.30%)


In [7]:
df['abs_magnetization'] = df['magnetization'].abs()

df_accepted = df[df['accepted'] == True].copy()

df_accepted['step'] = df_accepted.groupby('beta').cumcount()

fig = px.line(
    df_accepted,
    x='step',
    y='abs_magnetization',
    color='beta',
    labels={
        'step': 'Step',
        'abs_magnetization': 'Magnetization ⟨s⟩',
        'beta': 'β'
    },
    title='Magnetization evolution at different temperatures'
)

fig.update_layout(
    template='plotly_white',
    legend_title='β',
    width=1200,
    height=500
)

fig.show()